# Prediction of X-ray Diffraction Data Quality from Experimental Parameters
### Jesse Yoder
### April/May 2022
### The Data Inbuator

In [1]:
import glob
import os
import pandas as pd

In [2]:
#7 CSV files containing ~165k X-ray (only) structure IDs from PDB
%cat ./PDB_list_CSVs/rcsb_pdb_ids_000001-025000.txt \
| cut -c 1-49 


101D,101M,102D,102L,102M,103L,103M,104L,104M,105M


In [3]:
#Download all 165k structures 
#163,258 to be precise, April 11 2022 as query date
#I should make this a continous pipeline, where it finds the difference between the last files
#and pulls only what's new
#This is a dumb method - it just reads and pulls, no thinking

#to pull 165k structures took about 12 hours



#for file in `ls ./PDB_list_CSVs | grep rcsb_pdb_ids`; do ./PDB_batch_download.sh -f \
#./PDB_list_CSVs/$file -o ./PDB_depot -p; done


#test case with most recent structures (small set of 15k, less likely to be wierd)
#script from RCSB
#-f is file, -o is output dir, -p is get PDB  (-c cif, -a bioassembly, -s struct factors, etc)
#./PDB_batch_download.sh -f ./PDB_list_CSVs/rcsb_done_pdb_ids_150001-164657.txt -o ./PDB_depot/ -p

In [4]:
%%bash
#Create backup directory of PDB_depot
#Use pigz (*parallel* implementation of gzip) from homebrew
#runs in 5 minutes instead of hours

#tar -c --use-compress-program=pigz -f PDB_depot_archive.tar.gz ./PDB_depot/

In [5]:
%%bash
#I need to split the 165k pdb files to 9 subdirectories, based on first character
#Otherwise gunzip won't work (argument list too long)

#1st, make the 9 directories (eg. 1xxx_pdbs ... 9xxx_pdbs)
#for i in $(seq 1 9); do mkdir 'PDB_depot/'$i'xxx_pdbs'; done



In [6]:
"""
%%bash
#2nd, sort the PDB files, moving them to matching directory


for n in $(seq 1 9); do
    for i in `ls PDB_depot/${n}*.pdb.gz`; do    #backticks don't actually need curly braces for $VAR expansion
        mv $i  "PDB_depot/${n}xxx_pdbs"; #Double quotes do
    done
    done
    
"""

'\n%%bash\n#2nd, sort the PDB files, moving them to matching directory\n\n\nfor n in $(seq 1 9); do\n    for i in `ls PDB_depot/${n}*.pdb.gz`; do    #backticks don\'t actually need curly braces for $VAR expansion\n        mv $i  "PDB_depot/${n}xxx_pdbs"; #Double quotes do\n    done\n    done\n    \n'

In [7]:

"""

%%bash
#unzip everything
#force it to get rid of any test PDBs/duplicates
#no multi-core option for unzip, just go with usual

#Must pass specific file to gunzip, passing dir/* will result in "argument list too long"
for i in `ls PDB_depot | grep xxx_pdbs`; do
    for f in `ls ./PDB_depot/${i}/`; do 
        gunzip -d -f "./PDB_depot/${i}/${f}";
    #echo gunzip -d -f "./PDB_depot/${i}/*";
    done
    done
"""

'\n\n%%bash\n#unzip everything\n#force it to get rid of any test PDBs/duplicates\n#no multi-core option for unzip, just go with usual\n\n#Must pass specific file to gunzip, passing dir/* will result in "argument list too long"\nfor i in `ls PDB_depot | grep xxx_pdbs`; do\n    for f in `ls ./PDB_depot/${i}/`; do \n        gunzip -d -f "./PDB_depot/${i}/${f}";\n    #echo gunzip -d -f "./PDB_depot/${i}/*";\n    done\n    done\n'

In [21]:
%%timeit
#full PDB run took 4:30 hours! before optimizing. Now I would think 1-2 hours.
df = pd.DataFrame(columns=["Resolution", "Completeness", "I_sigma", "R_value", "R_free", "Detector", "Det_type", "Optics"])


#Add date - I can drop anything before... 2002? 20 years is a lot and the early entries are garbage/useless

                                    #only the PDBs IDs that start with 7 - small test
for filename in glob.glob('PDB_depot/*/*.pdb'):  #for full set, use wildcard to catch 9 subdirs 
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        
        resolution = completeness = i_sigma = r_value = r_free = detector = det_type = optics = "NULL"  #set as null in case line not present
        
        pdb_id = filename[20:24]
        for line in f:

            #Exit condition, 50% drop in time
            #works without strip? remove other line strips for speed 30% drop in time
            if line.startswith("ATOM"):
                break
            
            #Resolution
            if line.startswith("REMARK   3   RESOLUTION RANGE HIGH (ANGSTROMS)"):
                resolution = line.split(":")[1].strip()

            #Completeness
            if line.startswith("REMARK   3   COMPLETENESS FOR RANGE        (%)"):
                completeness = line.split(":")[1].strip()         
                
            #I/sigma
            if line.startswith("REMARK 200  <I/SIGMA(I)> FOR THE DATA SET"):
                i_sigma = line.split(":")[1].strip()      
            
            #R-Value
            if line.startswith("REMARK   3   R VALUE     (WORKING + TEST SET)"):
                r_value = line.split(":")[1].strip()
            
            
            #R-Free
            if line.startswith("REMARK   3   FREE R VALUE"):
                if line.split(":")[0].strip().endswith("VALUE"):  #need to match end here
                    r_free = line.split(":")[1].strip()
                    
                    
            #Detector Model (Pilatus 6M, Eiger 16M etc)
            if line.startswith("REMARK 200  DETECTOR MANUFACTURER"):
                detector = line.split(":")[1].strip()
                if ";" in detector:
                    detector = line.split(';')[0].strip()  #some lines contain 2 fields, 2nd redundant
                
            #Detector Type (Pixel, CCD, etc)
            if line.startswith("REMARK 200  DETECTOR TYPE"):
                det_type = line.split(":")[1].strip()

                    
            #Optics (Mirrors)
            if line.startswith("REMARK 200  OPTICS"):
                optics = line.split(":")[1].strip()

           


        df.loc[pdb_id] = [resolution, completeness, i_sigma, r_value, r_free, detector, det_type, optics]

KeyboardInterrupt: 

In [9]:
## Write dataframe to feather (can't have index)
# Loads about 100x faster than reading all the text files
df = df.reset_index()
df.to_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/1_initial_dataframe.feather')